In [1]:
import geopandas as gpd
import shapely as shp
import requests
import json
import pandas as pd
import numpy as np

#### This script consists out of 
- The generation of population data per location; 
- The generation of meteorological data per location; 
- The generation of elevation data per location

In [2]:
#choose here the shelter location that you want to retrieve data of
shelter_num = 2

In [3]:
# load input coordinates from a geodataframe (file) folder
folder_path = "data/voronoi_gdf"
voronoi_gdf = gpd.read_file(folder_path)
sportsfield_centers_gdf = gpd.read_file("data/sportsfield_centers_gdf", bbox=None, mask=None, rows=None, engine=None)

coords = sportsfield_centers_gdf['geometry']
coords = shp.get_coordinates(coords)
coords = coords.tolist()

coord = coords[shelter_num]
df = pd.DataFrame({'lon': [coord[0]], 'lat': [coord[1]], 'geometry': [coord]})
main_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

36.151255494244936


#### The generation of population data per location

In [4]:
from shelter import PopulationMapGenerator as pmg

In [5]:
# amount of persons for which each shelter is the closest (based on voronoi)
pop_file = 'data/population_tur_2020.csv'

map_generator = pmg(pop_file)
data = map_generator.filter_data()

population_gdf = map_generator.get_population_geo_df()
merged_data = gpd.sjoin(voronoi_gdf, population_gdf, how='inner', predicate='intersects')
population_sum = merged_data.groupby('geometry')['population_2020'].sum().reset_index()


population_sum_gdf = gpd.GeoDataFrame(population_sum, geometry='geometry', crs='EPSG:4326')

In [6]:
main_gdf = main_gdf.assign(population_2020=population_sum_gdf["population_2020"][shelter_num])

,lon,lat,geometry,population_2020
0,36.151255,36.188028,POINT (36.15126 36.18803),8999.194011


#### The generation of meteorological data per location

In [7]:
from shelter import WeatherData as wd

In [8]:
#retrieve weather data and append it location-specific
weather_data, weather_var = wd.get_weather_API(coord)
df_coord = wd.get_weather_data(coord, weather_data, weather_var)

{'temperature_2m_max': [21.62949297786241], 'temperature_2m_min': [13.052047131635325], 'rain_sum': [1.4431682932635086], 'snowfall_sum': [0.019753630088074264], 'windspeed_10m_max': [11.36568674125208], 'winddirection_10m_dominant': [209.98190906926922], 'max_temp': [34], 'min_temp': [23]}


In [9]:
main_gdf = main_gdf.join(df_coord, lsuffix='', rsuffix='r')

,lon,lat,geometry,population_2020,temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,windspeed_10m_max,winddirection_10m_dominant,max_temp,min_temp
0,36.151255,36.188028,POINT (36.15126 36.18803),8999.194011,21.629493,13.052047,1.443168,0.019754,11.365687,209.981909,34,23


#### The generation of elevation data per location

In [10]:
from shelter import SlopeFunctions as sf

In [11]:
folder_path = "data/filtered_sportsfield_gdf"
sportsfield_gdf = gpd.read_file(folder_path)

polygon_coords = []
for polygon in sportsfield_gdf['geometry']: 
    coords = (list(polygon.exterior.coords))
    polygon_coords.append(coords)

[[(36.12371739999999, 36.1933556), (36.124349699999996, 36.19387660000001), (36.125311, 36.1931168), (36.1246788, 36.1925958), (36.12371739999999, 36.1933556)], [(36.14869830000001, 36.187375800000005), (36.1489225, 36.1875755), (36.149305399999996, 36.1873004), (36.1490859, 36.187123199999995), (36.14869830000001, 36.187375800000005)], [(36.1509264, 36.187932), (36.15134139999999, 36.18830189999999), (36.1515846, 36.18812419999999), (36.151169599999996, 36.1877542), (36.1509264, 36.187932)], [(36.1476851, 36.184514500000006), (36.148258500000004, 36.18493899999999), (36.14853649999999, 36.18469439999999), (36.1479631, 36.1842699), (36.1476851, 36.184514500000006)], [(36.153183299999995, 36.2026731), (36.1535037, 36.20265069999999), (36.15346360000001, 36.202218099999996), (36.1531525, 36.202242899999995), (36.153183299999995, 36.2026731)], [(36.13531509999999, 36.17255), (36.135771399999996, 36.17268979999999), (36.13585719999999, 36.17250719999999), (36.13540090000001, 36.1723674), (

In [12]:
# Replace these coordinates with your latitude and longitude points

# Replace with the path to your GeoTIFF file
slopes = []
dem_file = 'data/n36_e036_1arc_v3.tif'

for coords in polygon_coords: 
    max_slope = sf.find_maximum_slope_among_points(coords, dem_file)
    slopes.append(max_slope)

num_slope = slopes[shelter_num]
main_gdf.assign(slope=num_slope)

c:\Users\maart\Documents\2 BUILDING TECHNOLOGY\CORE\SHELTER\shelter.py:64: RuntimeWarning: invalid value encountered in true_divide
  slope_radians = math.atan(delta_elevation / distance)


,lon,lat,geometry,population_2020,temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,windspeed_10m_max,winddirection_10m_dominant,max_temp,min_temp,slope
0,36.151255,36.188028,POINT (36.15126 36.18803),8999.194011,21.629493,13.052047,1.443168,0.019754,11.365687,209.981909,34,23,12.56627


In [15]:
main_gdf.to_csv('data/location_information.csv')

In [13]:
# # weather data for each input location (latitude, longitude)


# # open_meteo_url = []

# lat = []
# lon = []
# temps_high = []
# temps_low = []

# variables = ["temperature_2m_max", "temperature_2m_min","rain_sum,snowfall_sum","windspeed_10m_max","winddirection_10m_dominant"]
# mean_values = {var: [] for var in variables}

# for coord in coords: 
#     urllat = coord[0]
#     urllon = coord[1]

#     lat.append(urllat)
#     lon.append(urllon)
#     #set url for different datatypes on each location(temp max, temp min, rain, snow, wind, etc. editable; more options on the website) 
#     open_meteo ="http://archive-api.open-meteo.com/v1/archive?latitude=" +str(urllat) +str("&longitude=") +str(urllon) +str("&start_date=2022-01-01&end_date=2023-01-01&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,windspeed_10m_max,winddirection_10m_dominant&timezone=Africa/Cairo")
#     # open_meteo_url.append(open_meteo)


#     # for each location, request data from its url
#     response = requests.get(open_meteo)
#     data = response.json()
    
    
#     # Iterate through variables and calculate the mean for each
#     for var in variables:
#         daily_data = data['daily']
#         if var in daily_data:
#             daily_var = daily_data[var]
#             mean_value = np.mean(daily_var)
#             mean_values[var].append(mean_value)

# print(lat)
# print(lon)
# print(mean_values)

# # Create a DataFrame to store the data
# data = {'lat': lat, 'lon': lon, **mean_values}
# df = pd.DataFrame(data)

# # Create a GeoDataFrame from the DataFrame
# geometry = gpd.points_from_xy(df.lon, df.lat)
# gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
# print(gdf)
#     # data = (json.dumps(data, indent=4))
#     # df = pd.read_json(data)
#     # print(df.index)

#     # daily_temp_high = (df['daily'][1])
#     # daily_tem_low = (df['daily'][2])

#     # temp_average_high = np.mean(daily_temp_high)
#     # temps_high.append(temp_average_high)
#     # print(temp_average_high)

# # Getting the request library to load the data in json format


# # print all indices to see what data is available

# # try to do this for each index (in a loop??? would be nice)

# # location_info = gpd.sjoin(voronoi_gdf, population_gdf, how='inner', predicate='intersects')

